In [3]:
%matplotlib inline
%load_ext memory_profiler

import pandas as pd
import os
import glob
import pickle
import phate
import scprep
import meld
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import datetime
import scanpy as sc
from sklearn.decomposition import PCA
from py_pcha import PCHA
import math
import pegasus as pg
from scipy import sparse
import sys
sys.path.append('/home/ngr4/project/scripts/')
import utils

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [4]:
# load data
pdfp = '/home/ngr4/project/scnd/data/processed'
pfp = '/home/ngr4/project/scnd/results'

%memit adata = utils.load_adata(os.path.join(pdfp,'mouse_200614.h5ad'))

loaded @200617.13:13:09
took 18.67-s to load data
peak memory: 10887.28 MiB, increment: 10595.90 MiB


In [5]:
# load data that I have

def loadpkl(filename):
    with open(filename, 'rb') as f:
        datapkl = pickle.load(f)
    f.close()
    return datapkl

train = loadpkl(os.path.join(pdfp,'scnd_train_200529.pkl'))
test = loadpkl(os.path.join(pdfp,'scnd_test_200529.pkl'))
val = loadpkl(os.path.join(pdfp,'scnd_val_200529.pkl'))

dt = pd.DataFrame({'id':np.concatenate((np.concatenate((train['cell_id'], val['cell_id'])), test['cell_id'])),
                   'stored':np.concatenate((np.concatenate((train['louvain'], val['louvain'])), test['louvain'])),
                   'stored_ctype':np.concatenate((np.concatenate((train['ctype'], val['ctype'])), test['ctype']))})

del train, test, val

train2 = loadpkl(os.path.join(pdfp,'scnd_train_200528.pkl'))
test2 = loadpkl(os.path.join(pdfp,'scnd_test_200528.pkl'))
val2 = loadpkl(os.path.join(pdfp,'scnd_val_200528.pkl'))

dtt = pd.DataFrame({'id':np.concatenate((np.concatenate((train2['cell_id'], val2['cell_id'])), test2['cell_id'])),
                   'stored':np.concatenate((np.concatenate((train2['louvain'], val2['louvain'])), test2['louvain'])),
                   'stored_ctype':np.concatenate((np.concatenate((train2['ctype'], val2['ctype'])), test2['ctype']))})
del train2, test2, val2

old_train = loadpkl(os.path.join(pdfp, 'scnd_train_33p_200525.pkl'))
old_test = loadpkl(os.path.join(pdfp, 'scnd_test_7p_200525.pkl'))

dttt = pd.DataFrame({'id':np.concatenate((old_train['cell_id'], old_test['cell_id'])),
                   'stored':np.concatenate((old_train['louvain'], old_test['louvain'])),
                   'stored_ctype':np.concatenate((old_train['ctype'], old_test['ctype']))})
del old_train, old_test

d = pd.concat([dt, dtt, dttt]).drop_duplicates()

In [6]:
md = adata.obs.reset_index()
md = md.merge(d, right_on='id',left_on='index',how='inner')
# re-encode

In [7]:
for cid in md['louvain'].unique():
    print('\n----')
    series = md.loc[md['louvain']==cid,'stored_ctype'].value_counts()
    for i,ctype in enumerate(series.index):
        print('louvain: {}\tctype: {}\tN: {}'.format(cid,ctype,series[i]))
    print('----')


----
louvain: 18	ctype: Bergmann glia	N: 6498
louvain: 18	ctype: GABAergic interneuron 1	N: 14
louvain: 18	ctype: Astrocyte	N: 4
louvain: 18	ctype: GABAergic interneuron 2	N: 3
louvain: 18	ctype: Granule cell	N: 2
louvain: 18	ctype: Endothelial cell	N: 1
----

----
louvain: 0	ctype: Granule cell	N: 15225
louvain: 0	ctype: GABAergic interneuron 3	N: 1
----

----
louvain: 22	ctype: Astrocyte	N: 5506
louvain: 22	ctype: Bergmann glia	N: 29
louvain: 22	ctype: GABAergic interneuron 1	N: 12
louvain: 22	ctype: Oligodendrocyte	N: 4
louvain: 22	ctype: Endothelial cell	N: 3
louvain: 22	ctype: Pericyte	N: 1
louvain: 22	ctype: GABAergic interneuron 2	N: 1
----

----
louvain: 2	ctype: Granule cell	N: 12385
louvain: 2	ctype: GABAergic interneuron 1	N: 18
louvain: 2	ctype: Oligodendrocyte	N: 2
louvain: 2	ctype: Microglia	N: 2
louvain: 2	ctype: Astrocyte	N: 1
----

----
louvain: 20	ctype: Granule cell	N: 6066
----

----
louvain: 6	ctype: Granule cell	N: 10077
----

----
louvain: 16	ctype: Granule cell

In [8]:
louvain_encoder = {
    '0':'Granule cell',
    '1':'Granule cell',
    '2':'Granule cell',
    '3':'Granule cell',
    '4':'Granule cell',
    '5':'Granule cell',
    '6':'Granule cell',
    '7':'Granule cell',
    '8':'Granule cell',
    '9':'Granule cell',
    '10':'Granule cell',
    '11':'Granule cell',
    '12':'Granule cell',
    '13':'Granule cell',
    '14':'Granule cell',
    '15':'Granule cell',
    '16':'Granule cell',
    '17':'Granule cell',
    '18':'Bergmann glia',
    '19':'Oligodendrocyte',
    '20':'Granule cell',
    '21':'GABAergic interneuron 1',
    '22':'Astrocyte',
    '23':'GABAergic interneuron 1',
    '24':'GABAergic interneuron 2',
    '25':'Unipolar brush cell',
    '26':'Endothelial cell',
    '27':'Purkinje cell',
    '28':'Pericyte',
    '29':'Oligodendrocyte progenitor cell',
    '30':'GABAergic interneuron 3',
    '31':'Microglia',
}

In [9]:
# assign old cell type
stored_md = md.loc[:,['index','stored_ctype']].set_index('index')
adata.obs = adata.obs.merge(stored_md, left_index=True, right_index=True, how='left')
adata.obs['stored_ctype'] = adata.obs['stored_ctype'].fillna('?')
adata.obs['ctype'] = adata.obs['stored_ctype']

In [10]:
adata.obs['louvain'] = adata.obs['louvain'].astype(str)
adata.obs.loc[adata.obs['ctype']=='?', 'ctype'] = adata.obs.loc[adata.obs['ctype']=='?', 'louvain'].map(louvain_encoder)

In [11]:
# now save this
%memit utils.write_adata(os.path.join(pdfp,'mouse_200614.h5ad'), adata)

... storing 'louvain' as categorical
... storing 'stored_ctype' as categorical
... storing 'ctype' as categorical


saved @200617.13:14:34
took 15.26-s to save data
peak memory: 11083.18 MiB, increment: 0.04 MiB


In [12]:
adata.obs['ctype'].value_counts()

Granule cell                       260746
GABAergic interneuron 1             13709
Bergmann glia                        9595
Oligodendrocyte                      9375
Astrocyte                            8198
GABAergic interneuron 2              3899
Unipolar brush cell                  3154
Endothelial cell                     3033
Purkinje cell                        2174
Pericyte                             1502
Microglia                            1028
Oligodendrocyte progenitor cell      1007
GABAergic interneuron 3               892
Name: ctype, dtype: int64